# <a target="_blank" href="https://miptstats.github.io/courses/ad_mipt.html">Phystech@DataScience</a>


In [3]:
import numpy as np
import pandas as pd

import matplotlib.pyplot as plt
import seaborn as sns
sns.set(font_scale=1.3)

### Линейная регрессия: практика
Перед началом работы догрузим необходимые нам функции



In [4]:
from sklearn.datasets import load_diabetes
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression

Рассмотрим данные исследования 2004 года о зависимости риска развития диабета от пола, возраста, индекса массы тела, среднего кровяного давления и других показателей. 



#### Загрузка данных

In [7]:
data = load_diabetes()

Функция `sklearn.datasets.load_diabetes()` возвращает словарь. В поле `data` записана матрица регрессоров, в которой данные предварительно центрированы и нормированы. В поле `target` записана мера прогрессирования заболевания в течении года. В поле `DESCR` можно прочитать подробнее о данных.

Посмотрим на описание датасета.

In [10]:
print(data['DESCR'])

.. _diabetes_dataset:

Diabetes dataset
----------------

Ten baseline variables, age, sex, body mass index, average blood
pressure, and six blood serum measurements were obtained for each of n =
442 diabetes patients, as well as the response of interest, a
quantitative measure of disease progression one year after baseline.

**Data Set Characteristics:**

  :Number of Instances: 442

  :Number of Attributes: First 10 columns are numeric predictive values

  :Target: Column 11 is a quantitative measure of disease progression one year after baseline

  :Attribute Information:
      - age     age in years
      - sex
      - bmi     body mass index
      - bp      average blood pressure
      - s1      tc, total serum cholesterol
      - s2      ldl, low-density lipoproteins
      - s3      hdl, high-density lipoproteins
      - s4      tch, total cholesterol / HDL
      - s5      ltg, possibly log of serum triglycerides level
      - s6      glu, blood sugar level

Note: Each of these 1

Поле `data` содержит матрицу размера 442 $\times$ 10, где 442 — количество пациентов, а 10 — количество признаков (возраст, пол, и т.д.). 
Строки матрицы соответствуют пациентам, столбцы — признакам.

In [11]:
data['data'].shape

(442, 10)

Целевая переменная $-$ мера прогрессирования заболевания в течении года.

In [12]:
data['target'].shape

(442,)

In [20]:
data['data'][0][0]

0.038075906433423026

Создайте матрицу регрессоров $X$ (data) и столбец наблюдений $y$ (целевая переменная).

In [22]:
X, y = data['data'], data['target']

#### Обучение моделей

Разбейте данные случайно на две части — обучающую и тестовую в соотношении 80:20.

Если что-то забыли или что-то не понятно, можно почитать справку: https://scikit-learn.org/stable/modules/generated/sklearn.model_selection.train_test_split.html

При разбиении датасета стоит зафиксировать случайность для воспроизводимости результатов, поставив `random_state=42` в функцию разбиения (можете ставить любое число, необязательно именно 42)

In [23]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

print(X_train.shape, X_test.shape, y_train.shape, y_test.shape)

(353, 10) (89, 10) (353,) (89,)


Заведите модель линейной регрессии из `sklearn` и обучите ее по обучающей части данных.

In [26]:
model = LinearRegression().fit(X_train, y_train)

Посмотрите на результат обучения. Выведите коэффициенты перед признаками и свободный коэффициент.

In [30]:
print(model.coef_ , model.intercept_)

[  37.90402135 -241.96436231  542.42875852  347.70384391 -931.48884588
  518.06227698  163.41998299  275.31790158  736.1988589    48.67065743] 151.34560453985995


Выполните предсказание модели на тестовой выборке.

In [33]:
y_pred = model.predict(X_test)

Реализуйте метрики MSE, MAE, MAPE без использования `sklearn` и других готовых реализаций.

Пусть $Y_1, ..., Y_n$ &mdash; истинные значения, а $\widehat{Y}_1, ..., \widehat{Y}_n$ &mdash; предсказания.


Метрика MSE (mean squared error) определяется как
$$MSE = \frac{1}{n}\sum_{i=1}^n \left(Y_i - \widehat{Y}_i\right)^2.$$

Метрика **MAE** (*mean absolute error*), определяемая как 
$$MAE = \frac{1}{n}\sum_{i=1}^n \left|Y_i - \widehat{Y}_i\right|.$$

Метрика **MAPE** (*mean absolute percentage error*), определяемая как 
$$MAPE = 100\% \cdot \frac{1}{n}\sum_{i=1}^n \frac{\left|Y_i - \widehat{Y}_i\right|}{Y_i}.$$


In [35]:
def mean_squared_error(y_true, y_pred):
    n=len(y_true)
    res=np.sum((y_true-y_pred)**2)/n
    return res

def mean_absolute_error(y_true, y_pred):
    n=len(y_true)
    res=np.sum(abs((y_true-y_pred)))/n
    return res
    

def mean_absolute_percentage_error(y_true, y_pred):
    n=len(y_true)
    res=np.sum(abs((y_true-y_pred))/y_true)/n*100
    return res
    

Посчитайте MSE, MAE, MAPE на тестовой выборке и выведите с точностью до трех знаков после запятой.

In [40]:
round(mean_squared_error(y_test, y_pred),3)

2900.194

In [41]:
round(mean_absolute_error(y_test, y_pred),3)

42.794

In [42]:
round(mean_absolute_percentage_error(y_test, y_pred),3)

37.5

### Различные разбиения

Разбейте данные на тренировочную и тестовую выборки еще раз: измените пераметр `random_state`. Для каждого из разбиений обучите модель и получите метрики качества. Меняются ли эти метрики, если менять разбиение данных?

In [43]:
mse = []
mae = []
mape = []
for state in [1, 495, 324, 8, 0, 900]:

    # разбиение данных
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=state)
    
    # обучение модели
    model = LinearRegression().fit(X_train, y_train)

    # предсказание модели
    y_pred = model.predict(X_test)

    # подсчет метрики на тестовой выборке
    mse.append(round(mean_squared_error(y_test, y_pred),3))
    mae.append(round(mean_absolute_error(y_test, y_pred),3))
    mape.append(round(mean_absolute_percentage_error(y_test, y_pred),3))

In [47]:
mse, mae, mape

([2992.581, 2794.888, 3010.615, 2937.871, 3424.259, 2471.175],
 [41.975, 41.751, 45.098, 44.442, 46.174, 41.133],
 [32.494, 38.734, 41.314, 38.989, 38.045, 40.648])

Сравним корень из $MSE$ с $MAE$

In [50]:
print((np.sqrt(mse)))

[54.70448793 52.86670029 54.86907143 54.20213095 58.51716842 49.71091429]


**Вывод:** На данном семинаре мы научились работать с линейными моделями, исследовали ошибки предсказаний с помощью разных метрик на примере датасета с данными больных диабетом. По полученным данным (проще всего посмотреть на MAPE) можно сделать вывод о данная выборка не очень хорошо приближается линейной регерессией